In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uuuu/pytorch/default/1/CoreDischarge Model_epoch_4roc_0.7123073312793169.pth
/kaggle/input/los-ad/LOS/LOS_WEEKS_adm_val.csv
/kaggle/input/los-ad/LOS/LOS_WEEKS_adm_train.csv
/kaggle/input/los-ad/LOS/LOS_WEEKS_adm_test.csv
/kaggle/input/bert-of/LOS/LOS_WEEKS_adm_val.csv
/kaggle/input/bert-of/LOS/LOS_WEEKS_adm_train.csv
/kaggle/input/bert-of/LOS/LOS_WEEKS_adm_test.csv


In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoConfig
from torch.utils.data import DataLoader
from torch import nn
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import os
from torch.nn import functional as F
import torch.quantization
import torch.nn.utils.prune as prune

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1726773019.701679    1572 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
E0919 19:10:19.753869236    1572 oauth2_credentials.cc:238]            oauth_fetch: UN

# Loading Dataset

In [3]:
train=pd.read_csv("/kaggle/input/bert-of/LOS/LOS_WEEKS_adm_train.csv")
test=pd.read_csv("/kaggle/input/bert-of/LOS/LOS_WEEKS_adm_test.csv")
val=pd.read_csv("/kaggle/input/bert-of/LOS/LOS_WEEKS_adm_val.csv")

In [4]:
#train['los_label'][:5000].tolist()

# Maximum Length of a note

In [5]:
max(len(x) for x in train['text'])

22431

# Avg Length of a Note

In [ ]:
import statistics
statistics.mean([len(x) for x in train['text']])

### Model HyperParameters

In [8]:
batch_size = 8
max_tokens = 512
epochs = 200
best_roc_auc = 0.0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 3
gradient_accumulation_steps = 10
lr = 1e-5
temperature = 2.0
weight_decay = 0.01
num_warmup_steps = 0
pruning_ratio = 0.3
dropout_prob = 0.2

# Make Ensemble

In [9]:
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, input_ids, attention_mask):
        output1 = self.model1(input_ids, attention_mask=attention_mask)[0]
        output2 = self.model2(input_ids, attention_mask=attention_mask)[0]
        avg_output = (output1 + output2) / 2.00
        return avg_output

## Load Model From Hugging Face

In [10]:
config = AutoConfig.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1',
                                    num_labels=4,
                                    hidden_dropout_prob=dropout_prob,
                                    attention_probs_dropout_prob=dropout_prob)
core_model = AutoModelForSequenceClassification.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1' ,config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bvanaken/CORe-clinical-outcome-biobert-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
config = AutoConfig.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT',
                                    num_labels=4,
                                    hidden_dropout_prob=dropout_prob,
                                    attention_probs_dropout_prob=dropout_prob)

# Load the pre-trained model with the specified configuration
discharge_model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_Discharge_Summary_BERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Push Model To device ("Tpu")

> **Use Cuda For Mixed Precision Training** 

In [12]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    ^^^For MIXED PRECISION 
import torch_xla
import torch_xla.core.xla_model as xm

# Check if TPU is available
device = xm.xla_device()
print(device)

ensemble_model = EnsembleModel(core_model, discharge_model)
ensemble_model = ensemble_model.to(device)

E0000 00:00:1726773025.450449    1572 common_lib.cc:818] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:483


xla:0


# Creating custom dataset

In [13]:
tokenizer = AutoTokenizer.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1')

/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Dataset Class

In [14]:
class LosDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

### Tokenization

In [15]:
train_encodings = tokenizer(train['text'].tolist(),truncation=True, padding=True, max_length = max_tokens)                       #Expects list[str] so train['text'].tolist()
val_encodings = tokenizer(val['text'].tolist(), truncation=True, padding=True,  max_length = max_tokens)
test_encodings = tokenizer(test['text'].tolist(), truncation=True, padding=True , max_length = max_tokens)

In [16]:
train_dataset = LosDataset(train_encodings, train['los_label'].tolist())
val_dataset = LosDataset(val_encodings, val['los_label'].tolist())
test_dataset = LosDataset(test_encodings, test['los_label'].tolist())

In [17]:
len(train_dataset.labels[:])

30421

## Loading Model to Dataloader 

In [18]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Setting Up Optimizer And Scheduler

In [19]:
optimizer = AdamW(ensemble_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=len(train_loader) * epochs // gradient_accumulation_steps
)

/usr/local/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Loading Trained Model's state dict  (IF NEEDED)

### FOR MIXED PRECISION TRANING

In [ ]:
state_dict = torch.load('/kaggle/input/uuuu/pytorch/default/1/CoreDischarge Model_epoch_4roc_0.7123073312793169.pth')                 
# Load the state dict into your model
ensemble_model.load_state_dict(state_dict)
ensemble_model = ensemble_model.to(device) 

### FOR MULTI CORE TPU

In [20]:
state_dict = torch.load('/kaggle/input/uuuu/pytorch/default/1/CoreDischarge Model_epoch_4roc_0.7123073312793169.pth',map_location=torch.device('cpu')) # Pushing to Cpu
device = xm.xla_device()                  

# Load the state dict into your model
ensemble_model.load_state_dict(state_dict)
ensemble_model = ensemble_model.to(device)   # Pushing to TPU

/tmp/ipykernel_1572/3636163748.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/kaggle/input/uuuu/pytorch/default/1/CoreDischarge Model_epoch_4r

### Single Train Loader

In [21]:
for step,batch in enumerate(train_loader):
    print(step,end="\n \n")
    print(batch)
    
    if step==0:
        break

0
 
{'input_ids': tensor([[  101,  2705, 12522,  ...,  1113,  4870,   102],
        [  101,  2705, 12522,  ..., 19310,   131,   102],
        [  101,  2705, 12522,  ...,     0,     0,     0],
        ...,
        [  101,  2705, 12522,  ...,     0,     0,     0],
        [  101,  2705, 12522,  ...,   120,   185,   102],
        [  101,  2705, 12522,  ...,   117,  2999,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 2, 3, 1, 2, 0, 3, 2])}


# Train and Eval Loop


In [22]:
ensemble_name='Core+Discharge Model'
from tqdm import tqdm

## Dependiencies for TPU

In [23]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.core.functions as xf

### Model Parameters

In [24]:
ensemble_model.parameters

<bound method Module.parameters of EnsembleModel(
  (model1): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.2, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear

### See If Model loaded on Tpu

In [25]:
param_device = next(ensemble_model.parameters()).device
print(f"Model is loaded on device: {param_device}")

Model is loaded on device: xla:0


# **Using Multi Core  TPU for Training**

In [26]:
for epoch in range(epochs):
    ensemble_model.train()
    train_loss = 0
    para_loader = pl.MpDeviceLoader(train_loader, device)  #~~~~~~~~~~~~~~~~~~~~~  HERE
    for step, batch in enumerate(tqdm(para_loader)):       #~~~~~~~~~~~~~~~~~~~~~  HERE
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            xm.optimizer_step(optimizer)
            scheduler.step()

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    para_val_loader = pl.MpDeviceLoader(val_loader, device)       #~~~~~~~~~~~~~~~~~~~~~  HERE
    with torch.no_grad():
        for batch in tqdm(para_val_loader):                       #~~~~~~~~~~~~~~~~~~~~~  HERE
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())


    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')


    # Implement early stopping
    if macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0
        xm.save(ensemble_model.state_dict(), f"{ensemble_name}_epoch_{epoch}roc_{best_roc_auc}.pth")

100%|██████████| 549/549 [02:21<00:00,  3.88it/s]


Epoch: 1/200, Training Loss: 1.1207286196729243, Validation Loss: 1.165790316077095
Accuracy: 0.43976315190161697, Recall: 0.439763151901617, Precision: 0.4446875419560157, F1: 0.4272680911871791, Micro F1: 0.43976315190161697, Macro Roc Auc: 0.7149864563198599


100%|██████████| 549/549 [01:45<00:00,  5.20it/s]


Epoch: 2/200, Training Loss: 1.0965644086375852, Validation Loss: 1.1962702908368277
Accuracy: 0.440218629013892, Recall: 0.440218629013892, Precision: 0.45028747894311627, F1: 0.4305616015178639, Micro F1: 0.440218629013892, Macro Roc Auc: 0.7108677777112343
EarlyStopping counter: 1 out of 3


100%|██████████| 549/549 [01:47<00:00,  5.13it/s]


Epoch: 3/200, Training Loss: 1.065258313645321, Validation Loss: 1.1891659902093188
Accuracy: 0.42951491687542703, Recall: 0.42951491687542703, Precision: 0.4272291606057163, F1: 0.42020075082331787, Micro F1: 0.42951491687542703, Macro Roc Auc: 0.7112787973661283
EarlyStopping counter: 2 out of 3


100%|██████████| 549/549 [01:46<00:00,  5.14it/s]


Epoch: 4/200, Training Loss: 1.0343283285302862, Validation Loss: 1.204501924239004
Accuracy: 0.4345251651104532, Recall: 0.4345251651104532, Precision: 0.4380275976826215, F1: 0.42467038858247114, Micro F1: 0.4345251651104532, Macro Roc Auc: 0.7117125421949982
EarlyStopping counter: 3 out of 3
Early stopping


# **Using Mixed Precision Technique for Training**

### *Use Cuda for this*

In [ ]:
from torch.cuda.amp import autocast, GradScaler # IMPORTS


scaler = torch.amp.GradScaler("cuda")          #~~~~~~~~~~~~~~~~~~~~~  HERE

for epoch in range(epochs):
    
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Use autocast for mixed precision
        with torch.amp.autocast("cuda"):        #~~~~~~~~~~~~~~~~~~~~~  HERE
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss = loss / gradient_accumulation_steps

        # Scale the loss and call backward()
        scaler.scale(loss).backward()          #~~~~~~~~~~~~~~~~~~~~~  HERE
        train_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            # Unscales the gradients of optimizer's assigned params in-place
            scaler.unscale_(optimizer)
            
            # Clip gradients
            torch.nn.utils.clip_grad_norm_(ensemble_model.parameters(), max_norm=1.0)
            
            # Optimizer step 
            scaler.step(optimizer)            #~~~~~~~~~~~~~~~~~~~~~  HERE
            
            # Update the scale for next iteration
            scaler.update()
            scheduler.step()
            

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            #with torch.amp.autocast("cuda"):
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())

    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}') #,

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')
    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')

    # Implement early stopping
    if macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0
        torch.save(ensemble_model.state_dict(), f"{ensemble_name}_epoch_{epoch}roc_{best_roc_auc}.pth")

### Model evaluation on Test Dataset

In [27]:
#ensemble_model.to('cpu')
#device = 'cpu'

# Put the model in evaluation mode
ensemble_model.eval()

test_preds = []
test_labels = []

# Iterate over test data
para_test_loader = pl.MpDeviceLoader(test_loader, device)
with torch.no_grad():
    for batch in tqdm(para_test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

# Calculate metrics
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class, average='weighted')
precision = precision_score(test_labels, test_preds_class, average='weighted')
f1 = f1_score(test_labels, test_preds_class, average='weighted')
micro_f1 = f1_score(test_labels, test_preds_class, average='micro')
macro_roc_auc = roc_auc_score(test_labels, test_preds, multi_class='ovo', average='macro')

print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')

100%|██████████| 1100/1100 [1:05:36<00:00,  3.58s/it]

Accuracy: 0.4510628623394339, Recall: 0.4510628623394339, Precision: 0.4570243218028462, F1: 0.4405490711681504, Micro F1: 0.4510628623394339, Macro Roc Auc: 0.7177836144007887
